In [15]:
# 1. Read the ref-dist-mat into numpy for array
# 2. Replace array names with annotations
# 3. Double for loop to calculate F1 and F2
# 4. Use F measuremnts to calculated M and FST measures

In [16]:
import numpy as np

In [17]:
dist_mat = np.genfromtxt("./data/Clupea_11jan24/11jan_jc_ref-dist-mat.txt.txt", dtype="str")
annots = np.genfromtxt("./data/Clupea_11jan24/11jan_ClupeaAtmore_Fstannot.tsv", dtype="str")

# dist_mat = numpy.genfromtxt("/home/echarvel/Desktop/calab_data/code_bases/Skmer-2/fst_test_data/test_matrix.txt", dtype="str")
# annots = numpy.genfromtxt("/home/echarvel/Desktop/calab_data/code_bases/Skmer-2/fst_test_data/test_annots.txt", dtype="str")

In [18]:
for line in annots:
    x = np.where(dist_mat == line[0])
    for index in x: 
        try:
            dist_mat[index[0]][index[1]] = line[1]
        except: pass

In [19]:
dist_dict = dict()

for num1 in range(1, len(dist_mat[0])):
    for num2 in range(1, len(dist_mat[0])):
        if num1 != num2: 
            name1=dist_mat[0][num1]
            name2=dist_mat[0][num2]
            if name1 not in dist_dict:
                dist_dict[name1] = dict()

            if name2 not in dist_dict[name1]:
                dist_dict[name1][name2] = [float(dist_mat[num1, num2])]
            else: 
                dist_dict[name1][name2].append(float(dist_mat[num1, num2]))
        if num1 == num2:
            name1=name2=dist_mat[0][num1]
            
            if name1 not in dist_dict:
                dist_dict[name1] = dict()

            if name2 not in dist_dict[name1]:
                dist_dict[name1][name2] = []

key_list = list(dist_dict.keys())
print(key_list)

['Unknown', 'GieczPoland9th13thcentury', 'MalaNieszawkaPoland14th-15thc', 'TrusoJanówPomorskiPoland80085', 'KolowbrzwegBudzistowoPoland75', 'SelsoVestbyDenmark10th11thcen', 'SelsoVestbyDenmark12901380CE', 'KarmoyNorway2002', 'MoreNorway2002', 'RisorNorway2003', 'MaseskarSweden2003', 'KalixFinland2002', 'KalmarsundSweden2010', 'IdefjordInnerNorway2010', 'IdefjordOuterNorway2010']


In [20]:
def make_empty_matrix(dist_dict, key_list):
    matrix = [[0 for x in range(0, len(dist_dict)+1)] for x in range(0, len(dist_dict)+1)]
    matrix[0][0]="sample"
    for x in range(0, len(dist_dict)):
        matrix[0][x+1] = matrix[x+1][0] = (key_list[x])
    return matrix

def calculate_WC_FST(F1, F2, M):
    numerator_wc_fst = F1 + F2
    denominator_wc_fst = F1 + F2 + 2 * (1 / (M + 1)) * (M * (1 - F1) + (1 - F2))
    WC_FST = numerator_wc_fst / denominator_wc_fst
    return WC_FST

def calculate_Nei_FST(F1, F2):
    numerator_nei_fst = F1 + F2
    denominator_nei_fst = 2 - ((F1 + F2) / 2)
    Nei_FST = numerator_nei_fst / denominator_nei_fst
    return Nei_FST

def calculate_Hudson_FST(F1, F2):
    Hudson_FST = (F1 + F2) / 2
    return Hudson_FST

In [21]:
F_mtrx = make_empty_matrix(dist_dict, key_list)
wh_fst_mtrx = make_empty_matrix(dist_dict, key_list)
wc_fst_mtrx = make_empty_matrix(dist_dict, key_list)
nei_fst_mtrx = make_empty_matrix(dist_dict, key_list)
hud_fst_mtrx = make_empty_matrix(dist_dict, key_list)
clade_dist_mtrx = make_empty_matrix(dist_dict, key_list)
thetas_list = [[key,0.0] for key in key_list]
print(thetas_list[1][1])

0.0


In [22]:
for key1, index1 in zip(key_list, range(len(key_list))):
     for key2, index2 in zip(key_list, range(len(key_list))):
        if key1 == key2:
            break
        else:
            dist_array_k1_k1 = dist_dict[key1][key1]
            dist_array_k2_k2 = dist_dict[key2][key2]
            dist_array_k1_k2 = dist_dict[key1][key2]

            across_k1_k2 = sum(dist_array_k1_k2)/len(dist_array_k1_k2)

            max_k1_k1 = max(dist_array_k1_k1) if (len(dist_array_k1_k1) != 0) else 0
            max_k2_k2 = max(dist_array_k1_k1) if (len(dist_array_k1_k1) != 0) else 0
            max_k1_k2 = max(dist_array_k1_k2) 

            clade_dist_mtrx[index1+1][index2+1] = clade_dist_mtrx[index2+1][index1+1] = max_k1_k2 - (max_k1_k1 + max_k2_k2)/2
 
            if ((len(dist_array_k1_k1) == 0) or len(dist_array_k2_k2) == 0):
                # Checks if any of the populations only have one members.
                # Calculates F1 or F2 if possible. If not, assigns "NA". 
                # Assigns "NA" to all other Fst calculations.
                if (len(dist_array_k1_k1) != 0):
                    within_k1 = sum(dist_array_k1_k1)/len(dist_array_k1_k1)
                    F1 = within_k1/across_k1_k2
                    thetas_list[index1][1] = round(within_k1, 5)
                    F_mtrx[index1+1][index2+1] = round(F1, 5)
                    F_mtrx[index2+1][index1+1] = "NA"
                elif (len(dist_array_k2_k2) != 0):
                    within_k2 = sum(dist_array_k2_k2)/len(dist_array_k2_k2)
                    F2 = within_k2/across_k1_k2
                    thetas_list[index1][1] = "NA"
                    F_mtrx[index1+1][index2+1] = "NA"
                    F_mtrx[index2+1][index1+1] = round(F2, 5)
                    
                else:
                    F_mtrx[index1+1][index2+1] = F_mtrx[index2+1][index1+1] ="NA"

                wh_fst_mtrx[index1+1][index2+1] = wh_fst_mtrx[index2+1][index1+1] = "NA"
                wc_fst_mtrx[index1+1][index2+1] = wc_fst_mtrx[index2+1][index1+1] = "NA"
                nei_fst_mtrx[index1+1][index2+1] = nei_fst_mtrx[index2+1][index1+1] = "NA"
                hud_fst_mtrx[index1+1][index2+1] = hud_fst_mtrx[index2+1][index1+1] = "NA"
            else:
                within_k1 = sum(dist_array_k1_k1)/len(dist_array_k1_k1)
                within_k2 = sum(dist_array_k2_k2)/len(dist_array_k2_k2)

                F1 = within_k1/across_k1_k2
                F2 = within_k2/across_k1_k2

                if len(dist_array_k1_k1) <= len(dist_array_k2_k2):
                    M=len(dist_array_k1_k1)/len(dist_array_k2_k2)
                else:
                    M=len(dist_array_k2_k2)/len(dist_array_k1_k1)
                
                thetas_list[index1][1] = round(within_k1, 5)
                # NOTE: F_matrix contains F1 in the lower half and F2 in the upper half. 
                F_mtrx[index1+1][index2+1] =  round(F1, 5)
                F_mtrx[index2+1][index1+1] =  round(F2, 5)
                wh_fst_mtrx[index1+1][index2+1] = wh_fst_mtrx[index2+1][index1+1] = round((F1 + F2)/2, 5)
                wc_fst_mtrx[index1+1][index2+1] = wc_fst_mtrx[index2+1][index1+1] = round(calculate_WC_FST(F1, F2, M), 5)
                nei_fst_mtrx[index1+1][index2+1] = nei_fst_mtrx[index2+1][index1+1] = round(calculate_Nei_FST(F1, F2), 5)
                hud_fst_mtrx[index1+1][index2+1] = hud_fst_mtrx[index2+1][index1+1] =  round(calculate_Hudson_FST(F1, F2), 5)
for x in thetas_list:
    print(x)

['Unknown', 0.0]
['GieczPoland9th13thcentury', 0.02172]
['MalaNieszawkaPoland14th-15thc', 0.06176]
['TrusoJanówPomorskiPoland80085', 0.07604]
['KolowbrzwegBudzistowoPoland75', 'NA']
['SelsoVestbyDenmark10th11thcen', 0.13479]
['SelsoVestbyDenmark12901380CE', 0.06658]
['KarmoyNorway2002', 0.00245]
['MoreNorway2002', 'NA']
['RisorNorway2003', 0.00624]
['MaseskarSweden2003', 0.00283]
['KalixFinland2002', 0.00622]
['KalmarsundSweden2010', 0.00611]
['IdefjordInnerNorway2010', 0.00381]
['IdefjordOuterNorway2010', 0.01532]


In [23]:
np.savetxt("./ed_test_thetas.txt", np.matrix(thetas_list), fmt="%s", delimiter="\t")
np.savetxt("./ed_test_Fmat.txt", np.matrix(F_mtrx), fmt="%s", delimiter="\t")
np.savetxt("./ed_test_whfst.txt", np.matrix(wh_fst_mtrx), fmt="%s", delimiter="\t")
np.savetxt("./ed_test_wcfst.txt", np.matrix(wc_fst_mtrx), fmt="%s", delimiter="\t")
np.savetxt("./ed_test_neifst.txt", np.matrix(nei_fst_mtrx), fmt="%s", delimiter="\t")
np.savetxt("./ed_test_hudfst.txt", np.matrix(hud_fst_mtrx), fmt="%s", delimiter="\t")
np.savetxt("./ed_test_cladedist.txt", np.matrix(clade_dist_mtrx), fmt="%s", delimiter="\t")